In [7]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open
from urllib.request import urlopen

def readPDF(pdfFile):
    rsrcmgr = PDFResourceManager() #pdf 리소스 관리자
    retstr = StringIO() #pdf 입출력 객체
    laparams = LAParams() # 피라미터 객체
    #pdf=>text 변환기
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    process_pdf(rsrcmgr, device, pdfFile) # pdf=>text 변환 작업
    device.close() # 변환기 종료
    content = retstr.getvalue() #리턴된 스트링
    retstr.close()
    return content

#url 접속
pdfFile = urlopen(\
    'http://pythonscraping.com/pages/warandpeace/chapter1.pdf')
outputString = readPDF(pdfFile)
print(outputString)
pdfFile.close() #url 접속 종료
with open('c:/data/pdf/result.txt', 'w') as f: #쓰기 전용 파일
    f.write(outputString)
    print('저장되었습니다.')

CHAPTER I

"Well, Prince, so Genoa and Lucca are now just family estates of 
theBuonapartes. But I warn you, if you don't tell me that this 
means war,if you still try to defend the infamies and horrors 
perpetrated bythat Antichrist- I really believe he is Antichrist- I will 
havenothing more to do with you and you are no longer my friend, 
no longermy 'faithful slave,' as you call yourself! But how do you 
do? I seeI have frightened you- sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known 
AnnaPavlovna Scherer, maid of honor and favorite of the 
Empress MaryaFedorovna. With these words she greeted Prince 
Vasili Kuragin, a manof high rank and importance, who was the 
first to arrive at herreception. Anna Pavlovna had had a cough for 
some days. She was, asshe said, suffering from la grippe; grippe 
being then a new word inSt. Petersburg, used only by the elite.

All her invitations without exception, written in French, 
anddelivered by a scar

In [8]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
table = bs.find_all('table', {'class':'wikitable'})[0]
rows = table.find_all('tr')

csvFile = open('c:/data/csv/editors.csv', 'w', newline='', encoding='utf-8')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.find_all(['td', 'th']):
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    print('csv로 저장되었습니다.')
    csvFile.close()

csv로 저장되었습니다.


In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import datetime
import random
import pymysql
#mysql 접속
conn = pymysql.connect(host='localhost', port=3306, \
    user='web', password='1234', db='myweb', charset='utf8')
#sql을 실행하기 위해 cursor()사용
cursor = conn.cursor()
cursor.execute('use myweb')
cursor.execute('delete from pages')
#기준이 동일하면 랜덤 결괃 동일(재현성)
random.seed(datetime.datetime.now())


def store(title, content):
    title = title.replace("'", "''")
    content = content.replace("'", "''")
    sql=\
        "insert into pages (title, content) values ('%s', '%s')" % (title, content)

    cursor.execute(sql)
    conn.commit()


In [5]:
def getLinks(url):
    html = urlopen('http://ko.wikipedia.org'+url)
    obj = bs(html, 'html.parser')
    title = obj.find("h1").text

    content = obj.find('div', {'id':'mw-content-text'}).find('p').text
    store(title, content)
    return obj.find('div', \
        #정규표현식 ^ 시작, $ 끝, * 반복
        {'id': 'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    

In [6]:
links = getLinks('/wiki/%EC%9C%A4%EB%8F%99%EC%A3%BC')
try:
    cnt = 0
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links = getLinks(newArticle)
        cnt += 1
        if cnt > 50:
            break
finally:
    cursor.close()
    conn.close()
    print('completed')

/wiki/%EC%9E%84%EC%B9%98%EC%A0%95
/wiki/%EC%95%88%EC%B0%BD%ED%98%B8
/wiki/%EC%84%9C%EC%9E%AC%ED%95%84
/wiki/%EC%A3%BC%EC%8B%9C%EA%B2%BD
/wiki/1892%EB%85%84
/wiki/10%EC%9B%94_27%EC%9D%BC
/wiki/%EC%9E%90%ED%81%AC_%EB%8D%B0%EB%AF%B8
/wiki/%EC%9E%90%ED%81%AC_%EB%A6%AC%EB%B2%A0%ED%8A%B8
/wiki/%EB%88%84%EB%B2%A8_%EB%B0%94%EA%B7%B8
/wiki/%EB%84%A4_%EB%A9%8B%EB%8C%80%EB%A1%9C_%ED%95%B4%EB%9D%BC_(1960%EB%85%84_%EC%98%81%ED%99%94)
/wiki/%ED%94%84%EB%9E%91%EC%88%98%EC%95%84_%ED%8A%B8%EB%A4%BC%ED%8F%AC
/wiki/%EC%B9%B8_%EC%98%81%ED%99%94%EC%A0%9C
/wiki/%EB%B4%89%EC%A4%80%ED%98%B8
/wiki/%EC%BA%90%EC%84%9C%EB%A6%B0_%EB%B9%84%EA%B8%80%EB%A1%9C%EC%9A%B0
/wiki/%EC%BD%94%EC%96%B8_%ED%98%95%EC%A0%9C
/wiki/%EC%9C%8C%EB%A6%AC%EC%97%84_%EA%B3%A8%EB%93%9C%EB%A8%BC
/wiki/%EC%8A%A4%ED%8C%8C%EC%9D%B4%ED%81%AC_%EB%A6%AC
/wiki/%ED%86%A0%EB%93%9C_%ED%95%84%EB%93%9C
/wiki/%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4
/wiki/%EC%B2%B4%EB%A5%B4%EB%85%B8%EB%B9%8C
/